In [ ]:
from rekall.video_interval_collection_3d import VideoIntervalCollection3D
from rekall.interval_set_3d import Interval3D, IntervalSet3D
from vgrid_jupyter import VGridWidget
#from esper.widget import intervals_with_metadata
from query.models import FaceGender


def get_fps_map(vids):
    from query.models import Video
    vs = Video.objects.filter(id__in=vids)
    return {v.id: v.fps for v in vs}

def frame_second_conversion(c, mode='f2s'):
    from rekall.domain_interval_collection import DomainIntervalCollection
    from rekall.interval_set_3d import Interval3D
    fps_map = get_fps_map(set(c.get_grouped_intervals().keys()))
    
    def second_to_frame(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            t1,t2 = intrvl.t
            i2.t = (int(t1*fps), int(t2*fps))
            return i2
        return map_fn
    
    def frame_to_second(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            t1,t2 = intrvl.t
            i2.t = (int(t1/fps), int(t2/fps))
            return i2
        return map_fn
    
    if mode=='f2s':
        fn = frame_to_second
    if mode=='s2f':
        fn = second_to_frame
    output = {}
    for vid, intervals in c.get_grouped_intervals().items():
        output[vid] = intervals.map(fn(fps_map[vid]))
    return DomainIntervalCollection(output)

def frame_to_second_collection(c):
    return frame_second_conversion(c, 'f2s')

def second_to_frame_collection(c):
    return frame_second_conversion(c, 's2f')

In [ ]:
video = Video.objects.get(id=1)

In [ ]:
facegender = list(FaceGender.objects.filter(face__frame__video=video).select_related('face', 'face__frame', 'gender'))
intervals = [
    Interval3D(
        t=(f.face.frame.number / video.fps, (f.face.frame.number + 90) / video.fps),
        x=(f.face.bbox_x1, f.face.bbox_x2),
        y=(f.face.bbox_y1, f.face.bbox_y2),
        payload={'metadata': {
            'gender': {
                'type': 'Metadata_Categorical',
                'args': {
                    'category_type': 'gender',
                    'category': f.gender.id
                }
            }
        }, 'draw_type': {
            'type': 'DrawType_Bbox'
        }}
    )
    for f in facegender]

print(len(intervals))
vgrid_intervals = {'video_id': 1, 'interval_dict': {'test': IntervalSet3D(intervals).to_json()}}

In [ ]:
gender_colors = {
    Gender.objects.get(name='M').id: 'cyan',
    Gender.objects.get(name='F').id: 'red',
    Gender.objects.get(name='U').id: 'yellow',
}

VGridWidget(
    interval_blocks=[vgrid_intervals], 
    database={
        'videos': list(Video.objects.filter(id=video.id).values()),
        'gender': [{'color': gender_colors[g['id']], **g} for g in Gender.objects.all().values()]
    })

In [ ]:
from rekall.domain_interval_collection import DomainIntervalCollection
from rekall.interval_set_3d import IntervalSet3D, Interval3D
from rekall.vgrid_utils.vblocks_builder import VideoVBlocksBuilder, VideoTrackBuilder, DrawType_Bbox, Metadata_Categorical
from django.db.models import IntegerField, F
from django.db.models.functions import Cast

video = Video.objects.get(id=1)
qs = FaceGender.objects.filter(face__frame__video=video).select_related('face', 'face__frame', 'gender')

intervals = DomainIntervalCollection.from_django_qs(qs, schema={
    'domain': 'face.frame.video_id',
    't1': 'face.frame.number',
    't2': 'face.frame.number',
    'x1': 'face.bbox_x1', 'x2': 'face.bbox_x2', 'y1':'face.bbox_y1', 'y2':'face.bbox_y2',
    'payload': 'gender_id',
})
intervals = frame_to_second_collection(intervals).map(lambda i: Interval3D((i.t[0], i.t[1]+3), i.x, i.y, i.payload))

blocks = VideoVBlocksBuilder().add_track(
    VideoTrackBuilder('test', intervals
        ).set_draw_type(DrawType_Bbox()
        ).add_metadata('gender', Metadata_Categorical('gender'))
    ).build()

gender_colors = {
    Gender.objects.get(name='M').id: 'cyan',
    Gender.objects.get(name='F').id: 'red',
    Gender.objects.get(name='U').id: 'yellow',
}

VGridWidget(
    interval_blocks=blocks, 
    database={
        'videos': list(Video.objects.filter(id=video.id).values()),
        'gender': [{'color': gender_colors[g['id']], **g} for g in Gender.objects.all().values()]
    })

